In [19]:
from transformers import AutoModel, AutoConfig, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained('./output/checkpoint-87')
tokenizer = AutoTokenizer.from_pretrained('./onnx')

In [20]:
from onnxruntime import InferenceSession, SessionOptions, GraphOptimizationLevel

options = SessionOptions() # initialize session options
options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL

# 这里的路径传上一节保存的onnx模型地址
session = InferenceSession(
    "./onnx/model.onnx", sess_options=options, providers=["CPUExecutionProvider"]
)

# disable session.run() fallback mechanism, it prevents for a reset of the execution provider
session.disable_fallback() 


In [21]:
text = '股票在20200809的最高收盘价具体是多少？请保留X位小数？'
inputs = tokenizer(text, return_tensors='pt')
inputs = {k: v.detach().cpu().numpy() for k, v in inputs.items()}

# 运行
# 这里的logits要有export的时候output_names相对应
session.run(output_names=['logits'], input_feed=inputs)


[array([[-3.3771572,  3.8104706]], dtype=float32)]

In [22]:
import time

start = time.time()
session.run(output_names=['logits'], input_feed=inputs)
end = time.time() - start
end

0.020512104034423828